In [1]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch.nn as nn

# 🔄 Universal Dataset Class
class BinaryImageDataset(Dataset):
    def __init__(self, folder_path, label_map, split='train', val_split=0.2, img_size=(86, 86)):
        self.data = []
        self.labels = []
        self.img_size = img_size
        self.label_map = label_map

        for label in self.label_map:
            path = os.path.join(folder_path, label)
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.resize(img, self.img_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                self.data.append(img)
                self.labels.append(self.label_map[label])

        # Split
        X_train, X_val, y_train, y_val = train_test_split(
            self.data, self.labels, test_size=val_split, stratify=self.labels, random_state=42
        )

        if split == 'train':
            self.data = X_train
            self.labels = y_train
        else:
            self.data = X_val
            self.labels = y_val

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

# 📦 CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 10 * 10, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# 📁 Label map for yawning detection
label_map = {
    'no yawn': 0,
    'yawn': 1
}

# 🧠 Training Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_ds = BinaryImageDataset("/kaggle/input/yawn-dataset", label_map, split='train')
val_ds = BinaryImageDataset("/kaggle/input/yawn-dataset", label_map, split='val')

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 🚀 Training loop
for epoch in range(10):
    model.train()
    train_loss, train_correct = 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).sum().item()

    acc = train_correct / len(train_ds)
    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Accuracy: {acc:.4f}")

    # 🔍 Validation
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            val_correct += (outputs.argmax(1) == labels).sum().item()

    val_acc = val_correct / len(val_ds)
    print(f"Validation Accuracy: {val_acc:.4f}\n")

torch.save(model.state_dict(), "yawn_cnn_model.pth")


Epoch 1 - Train Loss: 41.7512, Accuracy: 0.8527
Validation Accuracy: 0.8867

Epoch 2 - Train Loss: 20.9195, Accuracy: 0.9402
Validation Accuracy: 0.9473

Epoch 3 - Train Loss: 15.5116, Accuracy: 0.9590
Validation Accuracy: 0.9590

Epoch 4 - Train Loss: 14.8119, Accuracy: 0.9580
Validation Accuracy: 0.9590

Epoch 5 - Train Loss: 10.6148, Accuracy: 0.9729
Validation Accuracy: 0.9717

Epoch 6 - Train Loss: 9.1186, Accuracy: 0.9785
Validation Accuracy: 0.9668

Epoch 7 - Train Loss: 8.4616, Accuracy: 0.9788
Validation Accuracy: 0.9727

Epoch 8 - Train Loss: 6.4558, Accuracy: 0.9832
Validation Accuracy: 0.9688

Epoch 9 - Train Loss: 5.7975, Accuracy: 0.9851
Validation Accuracy: 0.9707

Epoch 10 - Train Loss: 4.5890, Accuracy: 0.9888
Validation Accuracy: 0.9668

